Day 21 (https://adventofcode.com/2023/day/21)

In [1]:
import numpy as np

with open('./inputs/day21.txt', 'r') as f:
    garden_map = np.array([list(l.strip()) for l in f.readlines()])

    # I'll brute force part 1, knowing full well that probs won't work for part 2
    possibilities = [tuple(np.argwhere(garden_map=='S')[0])]
    for i in range(64):
        new_possibilities = []
        for p in possibilities:
            if p[0] > 0 and garden_map[(p[0]-1, p[1])] != '#':
                new_possibilities.append((p[0]-1, p[1]))
            if p[1] > 0 and garden_map[(p[0], p[1]-1)] != '#':
                new_possibilities.append((p[0], p[1]-1))
            if p[0]+1 < garden_map.shape[0] and garden_map[(p[0]+1, p[1])] != '#':
                new_possibilities.append((p[0]+1, p[1]))
            if p[1]+1 < garden_map.shape[1] and garden_map[(p[0], p[1]+1)] != '#':
                new_possibilities.append((p[0], p[1]+1))

        possibilities = list(set(new_possibilities))

    print('Answer to Day 21, Part 1:', len(possibilities))

    # very difficult
    # lot's of manual and pen-on-paper stuff that I don't feel like showing here,
    # but I got the answer
    part2_ans = 7424*(202300**2) + \
        7388*(202301**2) + \
        (933 + 941 + 948 + 956)*202300 - \
        (893 + 907 + 916 + 913)*202301

    print('Answer to Day 21, Part 2:', part2_ans)

Answer to Day 21, Part 1: 3646
Answer to Day 21, Part 2: 606188414811259


Day 22 (https://adventofcode.com/2023/day/22)

In [2]:
import numpy as np

with open('./inputs/day22.txt', 'r') as f:
    bricks = []
    for line in f.readlines():
        split_line = line.strip().split('~')
        bricks.append(
            {
                'x1': int(split_line[0].split(',')[0]),
                'y1': int(split_line[0].split(',')[1]),
                'z1': int(split_line[0].split(',')[2]),
                'x2': int(split_line[1].split(',')[0]),
                'y2': int(split_line[1].split(',')[1]),
                'z2': int(split_line[1].split(',')[2])
            }
        )

    brick_map = np.zeros(shape=(
        max(max(b['x1'], b['x2']) for b in bricks)+1,
        max(max(b['y1'], b['y2']) for b in bricks)+1,
        max(max(b['z1'], b['z2']) for b in bricks)+1,
    )) - 1

    for i, b in enumerate(bricks):
        brick_map[
            min(b['x1'], b['x2']) : max(b['x1'], b['x2'])+1,
            min(b['y1'], b['y2']) : max(b['y1'], b['y2'])+1,
            min(b['z1'], b['z2']) : max(b['z1'], b['z2'])+1,
        ] = i

    settled_brick_map = np.zeros(shape=brick_map.shape) - 1

    supported_by_dict = {}
    for i in sorted(range(len(bricks)), key = lambda x: min(bricks[x]['z1'], bricks[x]['z2'])):
        brick_blocks = np.argwhere(brick_map==i)
        underbrick_map = settled_brick_map[
            min(brick_blocks[:, 0]) : max(brick_blocks[:, 0])+1,
            min(brick_blocks[:, 1]) : max(brick_blocks[:, 1])+1,
            0 : min(brick_blocks[:, 2])
        ]
        nonempty_underbricks = np.argwhere(underbrick_map!=-1)
        m = len(nonempty_underbricks) and max(nonempty_underbricks[:, 2])

        supported_by_dict[i] = set(int(underbrick_map[tuple(x)]) for x in nonempty_underbricks if x[2] == m)
        settled_brick_map[
            min(brick_blocks[:, 0]) : max(brick_blocks[:, 0])+1,
            min(brick_blocks[:, 1]) : max(brick_blocks[:, 1])+1,
            m + 1 : max(brick_blocks[:, 2]) - min(brick_blocks[:, 2]) + m + 2
        ] = i

    single_point_of_failure_bricks = set(next(iter(b)) for b in supported_by_dict.values() if len(b)==1)
    print('Answer to Day 22, Part 1:', len(bricks) - len(single_point_of_failure_bricks))

    supporting_dict = {k : set(x for x in supported_by_dict if k in supported_by_dict[x]) for k in supported_by_dict}

    def get_num_falling(disintegrate_index):
        falling_bricks = [disintegrate_index]
        keep_going = True
        while keep_going:
            keep_going = False
            for falling_brick in falling_bricks:
                for potential_fall in supporting_dict[falling_brick]:
                    if potential_fall not in falling_bricks and all(b in falling_bricks for b in supported_by_dict[potential_fall]):
                        keep_going = True
                        falling_bricks.append(potential_fall)

        return len(falling_bricks) - 1
    
    print('Answer to Day 22, Part 2:', sum(get_num_falling(b) for b in single_point_of_failure_bricks))

Answer to Day 22, Part 1: 405
Answer to Day 22, Part 2: 61297


Day 23 (https://adventofcode.com/2023/day/23)

In [3]:
import numpy as np
import networkx as nx

with open('./inputs/day23.txt', 'r') as f:
    hike_map = np.array([list(l.strip()) for l in f.readlines()])
    hike_graph = nx.grid_2d_graph(n=hike_map.shape[0], m=hike_map.shape[1], create_using=nx.DiGraph)
    hike_graph.remove_nodes_from(map(tuple, np.argwhere(hike_map=='#')))

    for i, j in np.argwhere(hike_map=='v'):
        for x, y in hike_graph[(i, j)].copy():
            if i + 1 != x:
                hike_graph.remove_edge((i, j), (x, y))

    for i, j in np.argwhere(hike_map=='^'):
        for x, y in hike_graph[(i, j)].copy():
            if i - 1 != x:
                hike_graph.remove_edge((i, j), (x, y))

    for i, j in np.argwhere(hike_map=='>'):
        for x, y in hike_graph[(i, j)].copy():
            if j + 1 != y:
                hike_graph.remove_edge((i, j), (x, y))

    for i, j in np.argwhere(hike_map=='<'):
        for x, y in hike_graph[(i, j)].copy():
            if j - 1 != y:
                hike_graph.remove_edge((i, j), (x, y))

    simple_paths = list(nx.all_simple_paths(hike_graph, min(hike_graph),  max(hike_graph)))
    simple_paths.sort(key=len, reverse=True)

    print('Answer to Day 23, Part 1:', len(simple_paths[0])-1)

    simplified_graph = nx.grid_2d_graph(n=hike_map.shape[0], m=hike_map.shape[1], create_using=nx.Graph)
    simplified_graph.remove_nodes_from(map(tuple, np.argwhere(hike_map=='#')))
    for node in filter(lambda x: len(simplified_graph[x])==2, hike_graph):
        neighbors = list(simplified_graph.neighbors(node))
        simplified_graph.add_edge(
            neighbors[0],
            neighbors[1],
            weight=simplified_graph[node][neighbors[0]].get('weight', 1) + simplified_graph[node][neighbors[1]].get('weight', 1)
        )
        simplified_graph.remove_node(node)

    def longest_simple_path(graph, source, target, path_history=[], path_cost=0):
        path_history = path_history.copy()
        path_history.append(source)
        if source==target:
            return path_history, path_cost

        possible_paths = []
        for n in graph[source]:
            if n not in path_history:
                possible_paths.append(
                    longest_simple_path(
                        graph,
                        source=n,
                        target=target,
                        path_history=path_history,
                        path_cost=path_cost+graph[source][n]['weight']
                    )
                )

        possible_paths = list(filter(lambda x: x is not None, possible_paths))
        if len(possible_paths) > 0:
            return sorted(possible_paths, key=lambda x: x[1], reverse=True)[0]
        else:
            return None
        
    path, cost = longest_simple_path(simplified_graph, source=min(simplified_graph), target=max(simplified_graph))
    print('Answer to Day 23, Part 2:', cost)

Answer to Day 23, Part 1: 2386
Answer to Day 23, Part 2: 6246


Day 24 (https://adventofcode.com/2023/day/24)

In [4]:
import itertools
from mpmath import sin, cos, sqrt, linspace, mpf, matrix, pi, inf
from mpmath import mp
mp.dps = 25

def part1_intersection_point(hailstone1, hailstone2):
    m1 = hailstone1['vy']/hailstone1['vx']
    m2 = hailstone2['vy']/hailstone2['vx']
    b1 = hailstone1['py'] - m1*hailstone1['px']
    b2 = hailstone2['py'] - m2*hailstone2['px']

    if m1 == m2:
        assert b1 != b2, 'same line :-('
        return None
    
    x = (b1 - b2) / (m2 - m1)
    y = m1*x + b1

    if (x < hailstone1['px'] and hailstone1['vx'] > 0) or \
        (x < hailstone2['px'] and hailstone2['vx'] > 0) or \
        (x > hailstone1['px'] and hailstone1['vx'] < 0) or \
        (x > hailstone2['px'] and hailstone2['vx'] < 0):
        return None

    return x, y


with open('./inputs/day24.txt', 'r') as f:
    hailstones =  [
        {
            'px': int(l.split('@')[0].split(',')[0].strip()),
            'py': int(l.split('@')[0].split(',')[1].strip()),
            'pz': int(l.split('@')[0].split(',')[2].strip()),
            'vx': int(l.split('@')[1].split(',')[0].strip()),
            'vy': int(l.split('@')[1].split(',')[1].strip()),
            'vz': int(l.split('@')[1].split(',')[2].strip()),
        }
        for l in f.readlines()
    ]

    part1_ans = 0
    for h1, h2 in itertools.combinations(hailstones, 2):
        p = part1_intersection_point(h1, h2)
        if p is not None and \
            p[0] >= 200000000000000 and \
            p[1] >= 200000000000000 and \
            p[0] <= 400000000000000 and \
            p[1] <= 400000000000000:
            part1_ans += 1

    print('Answer to Day 24, Part 1:', part1_ans)

    class Vector:
        def __init__(self, x, y, z):
            self.x = mpf(x)
            self.y = mpf(y)
            self.z = mpf(z)

        def __repr__(self):
            return f'Vector({self.x}, {self.y}, {self.z})'

        def copy(self):
            return Vector(self.x, self.y, self.z)
        
        def __add__(self, other):
            return Vector(self.x+other.x, self.y+other.y, self.z+other.z)
        
        def __mul__(self, scalar):
            return Vector(scalar*self.x, scalar*self.y, scalar*self.z)
        
        def __sub__(self, other):
            return self + (other * -1)
        
        def __iter__(self):
            return iter((self.x, self.y, self.z))

        def rotate_radians(self, x_rot=0, y_rot=0, z_rot=0):
            x_rot_matrix = matrix([
                [mpf(1), mpf(0), mpf(0)],
                [mpf(0), cos(x_rot), sin(x_rot)],
                [mpf(0), -sin(x_rot), cos(x_rot)]
            ])

            y_rot_matrix = matrix([
                [cos(y_rot), mpf(0), -sin(y_rot)],
                [mpf(0), mpf(1), mpf(0)],
                [sin(y_rot), mpf(0),  cos(y_rot)]
            ])
            
            z_rot_matrix = matrix([
                [cos(z_rot), sin(z_rot), mpf(0)],
                [-sin(z_rot), cos(z_rot), mpf(0)],
                [mpf(0), mpf(0), mpf(1)]
            ])
            
            v = matrix([[self.x, self.y, self.z]])
            self.x, self.y, self.z = v * x_rot_matrix * y_rot_matrix * z_rot_matrix

            return self

    class Hailstone:
        def __init__(self, h):
            self.h = h
            self.p_vector = Vector(h['px'], h['py'], h['pz'])
            self.v_vector = Vector(h['vx'], h['vy'], h['vz'])

        def copy(self):
            return Hailstone(self.h)

        def rotate_radians(self, x, y, z):
            self.p_vector = self.p_vector.rotate_radians(x, y, z)
            self.v_vector = self.v_vector.rotate_radians(x, y, z)
            return self
        
        def at_time(self, t):
            return Vector(
                self.p_vector.x + t*self.v_vector.x,
                self.p_vector.y + t*self.v_vector.y,
                self.p_vector.z + t*self.v_vector.z
            )

    def intersection_point(hailstone1, hailstone2):
        m1 = hailstone1.v_vector.y / hailstone1.v_vector.x
        m2 = hailstone2.v_vector.y / hailstone2.v_vector.x
        b1 = hailstone1.p_vector.y - m1*hailstone1.p_vector.x
        b2 = hailstone2.p_vector.y - m2*hailstone2.p_vector.x

        if m1==m2:
            return None
        
        x = (b1 - b2) / (m2 - m1)
        y = m1*x + b1

        return x, y

    def dist(point1, point2):
        return sqrt(
            sum(
                (p1 - p2)**mpf(2) for p1, p2 in zip(point1, point2)
            )
        )

    def func(x, y, z):
        hail = [
            Hailstone(h).rotate_radians(x, y, z)
            for h in hailstones[55:60]
        ]
        intersections = [
            intersection_point(h1, h2)
            for h1, h2 in itertools.combinations(hail, 2)
        ]

        if any(i is None for i in intersections):
            return inf

        # sum of dists between intersection points
        return sum(dist(i1, i2) for i1, i2 in itertools.combinations(intersections, 2))

    def get_rotations(num_iterations=10):
        x, y, z = mpf(0), mpf(0), mpf(0)
        bound = pi
        for i in range(num_iterations):
            search_space = itertools.product(
                linspace(x-bound, x+bound, 20),
                linspace(y-bound, y+bound, 20),
                [0] # linspace(z-bound, z+bound, 20),
            )
            sweep = []
            for x, y, z in search_space:
                sweep.append((func(x, y, z), x, y, z))

            best, x, y, z = sorted(sweep, key=lambda x: x[0])[0]
            bound = bound / 8

        return x, y, z

    x_rot, y_rot, z_rot = get_rotations()

    intersection = intersection_point(
        Hailstone(hailstones[100]).rotate_radians(x_rot, y_rot, z_rot),
        Hailstone(hailstones[200]).rotate_radians(x_rot, y_rot, z_rot)
    )

    # rock estimate
    rock = Hailstone(
        {
            'px': intersection[0],
            'py': intersection[1],
            'pz': 0,
            'vx': 0,
            'vy': 0,
            'vz': 200
        }
    )

    rock.rotate_radians(0, 0, -z_rot)
    rock.rotate_radians(0, -y_rot, 0)
    rock.rotate_radians(-x_rot, 0, 0)

    def get_intersection_times(h1, h2, num_iterations=50):
        t1, t2 = mpf(0), mpf(0)
        bound = mpf(1000000000000000)
        for i in range(num_iterations):
            search_space = itertools.product(
                linspace(t1-bound, t1+bound, 50),
                linspace(t2-bound, t2+bound, 50),
            )
            sweep = []
            for t1, t2 in search_space:
                sweep.append((dist(h1.at_time(t1), h2.at_time(t2)), t1, t2))

            best, t1, t2 = sorted(sweep, key=lambda x: x[0])[0]
            bound = bound / 10

        return t1, t2

    h1 = Hailstone(hailstones[16])
    h2 = Hailstone(hailstones[14])
    t1 = round(get_intersection_times(rock, h1)[1])
    t2 = round(get_intersection_times(rock, h2)[1])

    vx_est = round((h2.at_time(t2).x - h1.at_time(t1).x) / (t2 - t1))
    vy_est = round((h2.at_time(t2).y - h1.at_time(t1).y) / (t2 - t1))
    vz_est = round((h2.at_time(t2).z - h1.at_time(t1).z) / (t2 - t1))
    px_est = h1.at_time(t1).x - t1*vx_est
    py_est = h1.at_time(t1).y - t1*vy_est
    pz_est = h1.at_time(t1).z - t1*vz_est

    # better rock estimate - I feel decently confident in the velocity vector especially
    rock = Hailstone(
        {
            'px': px_est,
            'py': py_est,
            'pz': pz_est,
            'vx': vx_est,
            'vy': vy_est,
            'vz': vz_est
        }
    )

    def closest_intersection(h1, h2, num_iterations=15):
        t = mpf(0)
        bound = mpf(1000000000000000)
        for i in range(num_iterations):
            sweep = []
            for t_test in linspace(t-bound, t+bound, 50):
                t_test = round(t_test)
                sweep.append((dist(h1.at_time(t_test), h2.at_time(t_test)), t_test))

            best, t = sorted(sweep, key=lambda x: x[0])[0]
            bound = bound / 10

        return best, t

    def find_start(h1, h2, rock, num_iterations=20):
        t1 = mpf(0)
        bound = mpf(1000000000000000)
        for i in range(num_iterations):
            sweep = []
            for t_test in linspace(t1-bound, t1+bound, 50):
                t_test = round(t_test)
                rock_start_p = h1.p_vector - ((rock.v_vector - h1.v_vector) * t_test)
                rock_est = Hailstone(
                    {
                        'px': rock_start_p.x,
                        'py': rock_start_p.y,
                        'pz': rock_start_p.z,
                        'vx': rock.v_vector.x,
                        'vy': rock.v_vector.y,
                        'vz': rock.v_vector.z,
                    }
                )
                best, t2 = closest_intersection(h2, rock_est)
                sweep.append((best, t_test, rock_est))

            best, t1, best_rock = sorted(sweep, key=lambda x: x[0])[0]
            bound = bound / 10
            if best == 0:
                break

        return best_rock

    h3 = Hailstone(hailstones[136])
    h4 = Hailstone(hailstones[124])

    r = find_start(h3, h4, rock)
    print('Answer to Day 24, Part 2:', int(sum(r.p_vector)))

Answer to Day 24, Part 1: 13149
Answer to Day 24, Part 2: 1033770143421619


Day 25 (https://adventofcode.com/2023/day/25)

In [5]:
import networkx as nx
import itertools

with open('./inputs/day25.txt', 'r') as f:
    connections = {l.split(':')[0] : l.split(':')[1].strip().split(' ') for l in f.readlines()}
    g = nx.Graph()
    for n1 in connections:
        for n2 in connections[n1]:
            if not g.has_edge(n1, n2):
                g.add_edge(n1, n2, capacity=1)

    for n1, n2 in itertools.combinations(g.nodes, 2):
        cut_value, groups = nx.minimum_cut(g, n1, n2)
        if cut_value == 3:
            break

    print('Answer to Day 25, Part 1:', len(groups[0])*len(groups[1]))
    print('Answer to Day 25, Part 2:', '❄️❄️❄️')

Answer to Day 25, Part 1: 552682
Answer to Day 25, Part 2: ❄️❄️❄️
